<a href="https://colab.research.google.com/github/anuga-community/anuga-clinic/blob/master/notebooks/notebook3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANUGA Clinic

## Notebook 3: Setting up a simple Erosion Operator 

Here we go through the process of creating an operator (fractional step operator) which implements a simple sand dune erosion operator. 

### Installation

These notebooks have been designed to run in the google `colaboratory` environment, which provides a jupyter notebook environment running on a virtual machine on the cloud. To use this environment you need a google account so that your notebook can be saved on google drive. 

To start interacting with the notebook follow the 
`View in Colaboratory` link above. 

## Setup Environment

If on github, first follow the link `View in Colaboratory' to start running on google's colab environment. Then ....

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory.

Wait until you see the comment *(4) Ready to go* before proceeding to subsequent commands. 

Th:e install should take less than a minute (and quicker if you have already run this earlier).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import sys

try:
  # On colab we can install all the packages we need via the notebook
  #
  # First download the clinic repository

  HOME = '/content'
  import os
  os.chdir(HOME)
  !git clone https://github.com/anuga-community/anuga-clinic.git

  # Now install environment using tool
  !/bin/bash /content/anuga-clinic/anuga_tools/install_anuga_colab.sh
 
except:
  from pathlib import Path
  HOME = str(Path.home())
                
%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

## Description

In this example we will demonstrate using the `Sanddune_erosion_operator` which was contibuteed to the anuga code by Ted Rigby. 

[The code is available from anuga github reposiory](https://github.com/stoiver/anuga_core/blob/master/anuga/operators/sanddune_erosion_operator.py)

We need to setup a quantity to represent the bedrock (elevation can erode down to the bedrock) and also the fractional step operator that does the erosion.

This tests the `sanddune_erosion_operator` confirming that:

- flow creates erosion when bed shear > critical and that erosion rates are higher in higher bed shear zones (typically higher velocity areas).
	
- that erosion is augmented by collapse of the sand face whenever erosion creates slopes > the angle of repose. This process leads to widening of the notch (laterally)and head like recession of  the main scour zone  as scour acts to steepen the longitudinal bed, triggering collapse of the steep bed and reshaping back to the angle of repose.
	
- that the operator can handle multiple erosion polygons with different base level.



### Import Modules


In [ ]:
import anuga
from anuga import Region, Geo_reference
import numpy as num


### Define topography as function of (relative) x and y

In [ ]:
def topography(x,y):
    """Complex topography defined by a function of vectors x and y."""
    print (' Creating topography....')
    
    z = 0.0*(x)                             # horizontal plane 

    N = len(x)
   
    for i in range(N):
        # First notched sand dune across Channel
        if 110 < x[i] <= 120:
            z[i] +=  1.0*(x[i]-110)       # Sloping U/S Face 1:1
        if 120 < x[i] < 130 :
            z[i] +=  10                   # Crest of Embankment at +10
        if 130 <= x[i] < 140:
            z[i] +=  10-1.0*(x[i] -130.0)  # Sloping D/S Face at 1:1

        # add notch in crest 1m wide by nom 300 deep
        # note sides are near vertical so will collapse back to repose even without erosion
        
        if 117 <= x[i] <= 133 and 2 <= y[i] <= 3:
            z[i] =  9                   # add 4m Notch in Embankment crest 
            
        # second lower plain sand dune across Channel
        if 230.0 < x[i] <= 240.0:
            z[i] +=  0.2*(x[i] -230)      # Sloping U/S Face 1:5        
        if 240 < x[i] < 250 :
            z[i] +=  2                    # Crest of Embankment at +2.0
        if 250 <= x[i] < 260:
            z[i] +=  2-0.2*(x[i] -250)   # Sloping D/S Face at 1:5      
    return z

### Setup computational domain

In [ ]:
x0 = 314036.58727982
y0 = 6224951.2960092
geo = Geo_reference(56, x0, y0, hemisphere='southern')

length = 360.
width = 5.
dx = dy = 1.0            # Resolution: Length of subdivisions on both axes
print ('>>>>> Domain has L = %f, W = %f, dx=dy= %f' %(length,width,dx) )
points, vertices, boundary = anuga.rectangular_cross(int(length/dx), int(width/dy), len1=length, len2=width)



domain = anuga.Domain(points, vertices, boundary, geo_reference = geo)
domain.set_flow_algorithm('DE0')
domain.set_name('sanddune') # Output name

domain.set_store_vertices_uniquely(False)

# Ensure elevation is stored and time varying
domain.set_quantities_to_be_stored({'elevation': 2,'stage': 2,'xmomentum': 2,'ymomentum': 2})

domain.set_quantity('elevation', topography, location='centroids')           # elevation is a function
domain.set_quantity('friction', 0.01, location='centroids')                  # Constant friction
domain.set_quantity('stage', expression='elevation', location='centroids')   # Dry initial condition

print (domain.statistics())

# Setup boundary conditions
Bi = anuga.Dirichlet_boundary([12.0,0.0,0.0]) # Inflow at depth
Br = anuga.Reflective_boundary(domain)              # Solid reflective side walls
Bo = anuga.Dirichlet_boundary([-5, 0, 0])           # uncontrolled outflow 

domain.set_boundary({'left': Bi, 'right': Bo, 'top': Br, 'bottom': Br})

### Set up a new quantity to represent the bedrock


In [ ]:
#--------------------------------------------
# build the check points and erosion polygons
#--------------------------------------------

polygon1    = num.array([ [106, 0.0], [144, 0.0], [144, 5.0], [106, 5.0] ])
polygon2    = num.array([ [230, 0.0], [260, 0.0], [260, 5.0], [230, 5.0] ])
poly1nsbase = 5  # set poly no scour base level in m
poly2nsbase = 3

# ma
polygon1 += num.array([x0,y0])
polygon2 += num.array([x0,y0])

# get the indices of triangles in each erosion poly so can setup quantity nsbase_ in domain
poly1ind = (Region(domain, polygon=polygon1)).indices
poly2ind = (Region(domain, polygon=polygon2)).indices
print ('>>>>> poly1ind is of length ', len(poly1ind), ' and contains triangles', poly1ind[0], ' to ' , poly1ind[-1])
print ('>>>>> poly2ind is of length ', len(poly2ind), ' and contains triangles', poly2ind[0], ' to ' , poly2ind[-1])

# get the initial model surface elevation
nsbase_elev_c = domain.get_quantity('elevation').get_values(location='centroids')
print ('>>>>> nsbase_elev_c is of length ',len(nsbase_elev_c) )


# build the no scour base surface  by combining initial elev where < nsbase and nsbase in each scour poly
nsbase_elev_c[poly1ind] = num.minimum(nsbase_elev_c[poly1ind], poly1nsbase)
nsbase_elev_c[poly2ind] = num.minimum(nsbase_elev_c[poly2ind], poly2nsbase)


# create new Anuga quantity and assign nsbase_elev_c values to domain so can distribute in parallel code
anuga.Quantity(domain, name='nsbase_elevation', register=True)
domain.set_quantity('nsbase_elevation',nsbase_elev_c, location='centroids')

# Print out polygon points and ids as check
print ('>>>>> erosion polygon1 contains ', polygon1)
print ('>>>>> erosion polygon2 contains ', polygon2)

### Setup sanddune erosion operator 


In [ ]:
from anuga import Sanddune_erosion_operator

# assign ns base elevations to the domain
nsbase_elev_c = domain.get_quantity('nsbase_elevation').get_values(location='centroids')

poly1ind = (Region(domain, polygon=polygon1)).indices
poly2ind = (Region(domain, polygon=polygon2)).indices
indices_union = list(set(poly1ind) | set(poly2ind))

# setup and create operator within polys setting the scour base elevations for each poly
op0 = Sanddune_erosion_operator(domain, base=nsbase_elev_c, indices=indices_union, Ra=45)   # both dunes

### Evolve simulation through time  

In [ ]:
for t in domain.evolve(yieldstep=2.0, duration=100.0):
    domain.print_timestepping_statistics()
    

#  run completed - tidy up 
print (' >>>>> Simulation completed successfully ')

## Visualise Flow

Let's open up the `swwfile` and look at the evolution of the `depth` the `stage` and the `elev`.

In [ ]:
# Create a wrapper for contents of sww file
swwfile = 'sanddune.sww'
splotter = anuga.SWW_plotter(swwfile)



#### Plot stage


In [ ]:
splotter.triang.set_mask(None)
for i,time in enumerate(splotter.time):
  #print (time)
  splotter.save_stage_frame(frame=i)
  
splotter.make_stage_animation()

#### Plot depth


In [ ]:
splotter.triang.set_mask(None)
for i,time in enumerate(splotter.time):
  #print (time)
  splotter.save_depth_frame(frame=i)
  
splotter.make_depth_animation()

#### 3D visualisation

We can use the `mpl` module to produce some 3D plots. We need to create vertex values given the centroid values.

This code produes an extra extraneous plot (any suggestion to fix that is very welcome).

In [ ]:
X = np.vstack((splotter.xc, splotter.yc)).transpose()
E = splotter.elev

splotter.nodes = np.vstack((splotter.x,splotter.y)).transpose()
import scipy

stage_v = []
for i,S in enumerate(splotter.stage):
  #print (i, S.shape, X.shape)
  Interp = scipy.interpolate.NearestNDInterpolator(X,S)
  stage_v.append(Interp(splotter.nodes))
  
stage_v = np.array(stage_v)

elev_v = []
for i,E in enumerate(splotter.elev):
  #print i, E.shape, X.shape
  Interp = scipy.interpolate.NearestNDInterpolator(X,E)
  elev_v.append(Interp(splotter.nodes))  

elev_v = np.array(elev_v)

from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure()
#fig, ax = plt.subplots()
ax = plt.gca(projection='3d')

def update_image(i):
    ax.clear()
    ax.plot_trisurf(splotter.triang, elev_v[i*step], color='navajowhite', edgecolors='none', antialiased=False, shade=True)
    ax.plot_trisurf(splotter.triang, stage_v[i*step], color='dodgerblue', edgecolors='none', alpha=1.0, linewidth=0, antialiased=False, shade=True)
    ax.set_zlim(elev_v.min(), np.max([stage_v.max(), elev_v.max()]))
    

# increase step to skip frames; increase interval to increase movie speed
step = 4
anim = animation.FuncAnimation(fig, update_image, frames=len(splotter.depth)//step, interval=20);

anim

#### Examine elevation at points

As in the previous notebook, it is instructive to examine the value of the quantity of interest at specific locations. 

In [ ]:
# Alias
xc = splotter.xc
yc = splotter.yc

# Observation points across the first step
point_observations = [[125.3, 0.0], 
                      [125.3, 0.5], 
                      [125.3, 1.0], 
                      [125.3, 1.5],
                      [125.3, 2.0],
                      [125.3, 2.5],
                     ]

# Find nearest centroid to observation points
nearest_points = []
for row in point_observations:
    nearest_points.append(np.argmin( (xc-row[0])**2 + (yc-row[1])**2 ))
    
fig, ax = plt.subplots()

for i, npt in enumerate(nearest_points):
  ax.plot(splotter.time, splotter.elev[:,npt], label=str(point_observations[i][1]) )

ax.legend(loc = 'center right')
plt.title('Elevation')
plt.xlabel('time')
plt.ylabel('height')




print (nearest_points)
 